# Fully Convolutional Watershed Distance Transform for 2D Data
---
Implementation of papers:

[Deep Watershed Transform for Instance Segmentation](http://openaccess.thecvf.com/content_cvpr_2017/papers/Bai_Deep_Watershed_Transform_CVPR_2017_paper.pdf)

[Learn to segment single cells with deep distance estimator and deep cell detector](https://arxiv.org/abs/1803.10829)

In [1]:
import os
import errno

import numpy as np

import sys
sys.path.insert(0,"/home/snn/deepcell-tf")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)


import deepcell

In [2]:
import math
import numpy as np
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import callbacks
from tensorflow.python.keras.optimizers import SGD

from tensorflow.python.keras.utils.data_utils import get_file

from deepcell import losses
import image_gen as image_gen
from deepcell import image_generators
from deepcell import model_zoo
from deepcell.layers import TensorProduct, ReflectionPadding3D, DilatedMaxPool3D

from deepcell.utils import train_utils
from deepcell.utils.data_utils import get_data
from deepcell.utils.train_utils import rate_scheduler
from deepcell.training import train_model_conv


from tensorflow.python.keras.layers import MaxPool3D
from conv_gru_layer import ConvGRU2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout, LeakyReLU
from tensorflow.python.keras.layers import Conv3D, ZeroPadding3D, ConvLSTM2D, Cropping3D
from tensorflow.python.keras.layers import Input, Add, Concatenate, Flatten, Reshape
from tensorflow.python.keras.engine.input_layer import InputLayer

from tensorflow.python.keras.models import Model


from tensorflow.python.keras.regularizers import l2
from deepcell.layers import ImageNormalization2D, ImageNormalization3D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Softmax

from sklearn.model_selection import train_test_split

from tensorflow.python.client import device_lib

### Load the Training Data

In [ ]:
# If DATA_FILE exists just run this cell

from deepcell.utils.data_utils import get_data

DATA_DIR = '/data/npz_data/tracking_benchmark_data'
DATA_FILE = os.path.join(DATA_DIR, '3T3_HeLa_HEK_RAW_cropped.npz')

# Load Information for hardcoded image size training
seed = 1
test_size = .2
train_dict, test_dict = get_data(DATA_FILE, seed=seed, test_size=test_size)
X_train, y_train = train_dict['X'], train_dict['y']
X_test, y_test = test_dict['X'], test_dict['y']

print(' -\nX.shape: {}\ny.shape: {}'.format(train_dict['X'].shape, train_dict['y'].shape))

### Set up filepath constants

In [ ]:
# Set up other required filepaths
PREFIX = os.path.relpath(os.path.dirname(DATA_FILE), DATA_DIR)
ROOT_DIR = '/data'  # TODO: Change this! Usually a mounted volume
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models', PREFIX))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs', PREFIX))

### First, create a foreground/background separation model

#### Instantiate the fgbg model

In [ ]:
def feature_net_3D(receptive_field=61,
                    n_frames=3,
                    input_shape=(5, 256, 256, 1),
                    n_features=3,
                    n_channels=1,
                    reg=1e-5,
                    n_conv_filters=64,
                    n_dense_filters=200,
                    gru_kernel_size =3,
                    VGG_mode=False,
                    init='he_normal',
                    norm_method='whole_image',
                    gru=False,
                    location=False,
                    dilated=False,
                    padding=False,
                    padding_mode='reflect',
                    multires=False,
                    include_top=True):
    # Create layers list (x) to store all of the layers.
    # We need to use the functional API to enable the multiresolution mode
    x = []

    win = (receptive_field - 1) // 2
    win_z = (n_frames - 1) // 2

    if dilated:
        padding = True

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
        time_axis = 2
        row_axis = 3
        col_axis = 4
        if not dilated:
            input_shape = (n_channels, n_frames, receptive_field, receptive_field)
    else:
        channel_axis = -1
        time_axis = 1
        row_axis = 2
        col_axis = 3
        if not dilated:
            input_shape = (n_frames, receptive_field, receptive_field, n_channels)

    x.append(Input(shape=input_shape))
    x.append(ImageNormalization3D(norm_method=norm_method, filter_size=receptive_field)(x[-1]))

    if padding:
        if padding_mode == 'reflect':
            x.append(ReflectionPadding3D(padding=(win_z, win, win))(x[-1]))
        elif padding_mode == 'zero':
            x.append(ZeroPadding3D(padding=(win_z, win, win))([-1]))

    if location:
        x.append(Location3D(in_shape=tuple(x[-1].shape.as_list()[1:]))(x[-1]))
        x.append(Concatenate(axis=channel_axis)([x[-2], x[-1]]))

    if multires:
        layers_to_concat = []

    rf_counter = receptive_field
    block_counter = 0
    d = 1

    append_gru = False
    while rf_counter > 4:
        filter_size = 3 if rf_counter % 2 == 0 else 4
        if append_gru == True:
            x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(filter_size, filter_size), dilation_rate=(d, d),
                            padding='valid', kernel_initializer=init,
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        else:
            x.append(Conv3D(n_conv_filters, (1, filter_size, filter_size), 
                            dilation_rate=(1, d, d), kernel_initializer=init,
                            padding='valid', kernel_regularizer=l2(reg))(x[-1]))

        
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        x.append(Activation('relu')(x[-1]))

        block_counter += 1
        rf_counter -= filter_size - 1

        if block_counter % 2 == 0:
            if dilated:
                x.append(DilatedMaxPool3D(dilation_rate=(1, d, d), pool_size=(1, 2, 2))(x[-1]))
                d *= 2
            else:
                x.append(MaxPool3D(pool_size=(1, 2, 2))(x[-1]))

            if VGG_mode:
                n_conv_filters *= 2

            rf_counter = rf_counter // 2

            if multires:
                layers_to_concat.append(len(x) - 1)

    if multires:
        c = []
        for l in layers_to_concat:
            output_shape = x[l].get_shape().as_list()
            target_shape = x[-1].get_shape().as_list()
            time_crop = (0, 0)

            row_crop = int(output_shape[row_axis] - target_shape[row_axis])

            if row_crop % 2 == 0:
                row_crop = (row_crop // 2, row_crop // 2)
            else:
                row_crop = (row_crop // 2, row_crop // 2 + 1)

            col_crop = int(output_shape[col_axis] - target_shape[col_axis])

            if col_crop % 2 == 0:
                col_crop = (col_crop // 2, col_crop // 2)
            else:
                col_crop = (col_crop // 2, col_crop // 2 + 1)

            cropping = (time_crop, row_crop, col_crop)

            c.append(Cropping3D(cropping=cropping)(x[l]))
        x.append(Concatenate(axis=channel_axis)(c))
        
    
    x.append(Conv3D(n_dense_filters, (1, rf_counter, rf_counter), dilation_rate=(1, d, d), kernel_initializer=init, padding='valid', kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))

    x.append(Conv3D(n_dense_filters, (n_frames, 1, 1), dilation_rate=(1, d, d), kernel_initializer=init, padding='valid', kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))
    
    if gru == True:
        x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(gru_kernel_size, gru_kernel_size),
                            padding='same', kernel_initializer=init, activation='relu',
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        # x.append(Activation('relu')(x[-1]))
        x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(gru_kernel_size +2, gru_kernel_size +2),
                            padding='same', kernel_initializer=init, activation='relu',
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        # x.append(Activation('relu')(x[-1]))
    
    
    x.append(TensorProduct(n_dense_filters, kernel_initializer=init, kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))

    x.append(TensorProduct(n_features, kernel_initializer=init, kernel_regularizer=l2(reg))(x[-1]))

    if not dilated:
        x.append(Flatten()(x[-1]))

    if include_top:
        x.append(Softmax(axis=channel_axis)(x[-1]))

    model = Model(inputs=x[0], outputs=x[-1])
    model.summary()

    return model

In [ ]:
def feature_net_skip_3D(receptive_field=61,
                        input_shape=(5, 256, 256, 1),
                        fgbg_model=None,
                        gru=False,
                        gru_kernel_size=3,
                        last_only=True,
                        n_skips=1,
                        norm_method='whole_image',
                        padding_mode='reflect',
                        **kwargs):
    # print(K.image_data_format())
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    inputs = Input(shape=input_shape)
    
    img = ImageNormalization3D(norm_method=norm_method,
                               filter_size=receptive_field)(inputs)

    models = []
    model_outputs = []

    if fgbg_model is not None:
        for layer in fgbg_model.layers:
            layer.trainable = False
        models.append(fgbg_model)
        fgbg_output = fgbg_model(inputs)
        if isinstance(fgbg_output, list):
            fgbg_output = fgbg_output[-1]
        model_outputs.append(fgbg_output)

    for _ in range(n_skips + 1):
        if model_outputs:
            model_input = Concatenate(axis=channel_axis)([img, model_outputs[-1]])
        else:
            model_input = img
        new_input_shape = model_input.get_shape().as_list()[1:]
        models.append(feature_net_3D(receptive_field=receptive_field, 
                                     input_shape=new_input_shape, norm_method=None, dilated=True, 
                                     padding=True, padding_mode=padding_mode, gru=gru, 
                                     gru_kernel_size=gru_kernel_size, **kwargs))
        model_outputs.append(models[-1](model_input))

    if last_only:
        model = Model(inputs=inputs, outputs=model_outputs[-1])
    else:
        if fgbg_model is None:
            model = Model(inputs=inputs, outputs=model_outputs)
        else:
            model = Model(inputs=inputs, outputs=model_outputs[1:])
    model.summary()
    return model

#### Test Generator

In [ ]:
DataGenerator = image_generators.MovieDataGenerator

datagen = DataGenerator(
    rotation_range=180,
    shear_range=False,
    zoom_range=(0.8, 1.2),
    horizontal_flip=True,
    vertical_flip=True)

datagen_val = DataGenerator(
    rotation_range=0,
    shear_range=0,
    zoom_range=0,
    horizontal_flip=0,
    vertical_flip=0)


train_data = datagen.flow(
    train_dict,
    skip=3,
    seed=0,
    batch_size=1,
    transform='fgbg',
    transform_kwargs={},
    frames_per_batch=3)

val_data = datagen_val.flow(
    test_dict,
    skip=3,
    seed=0,
    batch_size=1,
    transform='fgbg',
    transform_kwargs={},
    frames_per_batch=3)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

print(train_data.x.shape)

index = np.random.randint(low=0, high=train_data.x.shape[0])
frame = np.random.randint(low=0, high=train_data.x.shape[1])
print('Image number:', index)
print('Frame number:', frame)

fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(10, 5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(train_data.x[index, frame, ..., 0])
ax[0].set_title('Source Image X')

ax[1].imshow(train_data.y[index, frame, ..., 1])
ax[1].set_title('Source Image Y')

fig.tight_layout()
plt.show()

#### (1) Function to create the model fgbg model

In [ ]:
def create_and_train_fgbg(lr_sched, train_dict, test_dict, fgbg_gru_model_name, gru_kernel_size=3, gru=False):
    
    fgbg_model = feature_net_skip_3D(
                    n_features=2,  # segmentation mask (is_cell, is_not_cell)
                    receptive_field=receptive_field,
                    n_skips=n_skips,
                    n_conv_filters=32,
                    n_dense_filters=128,
                    gru=gru,
                    gru_kernel_size=gru_kernel_size,
                    input_shape=tuple([frames_per_batch] + list(train_dict['X'].shape[2:])),
                    last_only=False)

    print("Training fgbg model. \n")

    fgbg_model = train_model_conv(
        model=fgbg_model,
        train_dict=train_dict,
        test_dict=test_dict,
        model_name=fgbg_gru_model_name,
        log_dir=LOG_DIR,
        transform='fgbg',
        optimizer=optimizer,
        batch_size=batch_size,
        frames_per_batch=frames_per_batch,
        n_epoch=n_epoch,
        model_dir=MODEL_DIR,
        lr_sched=lr_sched,
        rotation_range=180,
        flip=True,
        shear=False,
        zoom_range=(0.8, 1.2))

    # Save model
    fgbg_gru_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(fgbg_gru_model_name))
    fgbg_model.save_weights(fgbg_gru_weights_file)

    return fgbg_model

### Next, Create a model for the watershed energy transform

### Function to create the distance transform model

In [ ]:
def create_and_train_gru(lr_sched, train_dict, test_dict, conv_gru_model_name, gru_kernel_size=3, gru=False):
    
    conv_gru_model = feature_net_skip_3D(
        receptive_field=receptive_field,
        n_skips=n_skips,
        n_features=4,  # (background edge, interior edge, cell interior, background)
        n_frames=frames_per_batch,
        n_conv_filters=32,
        n_dense_filters=128,
        gru=gru,
        gru_kernel_size=gru_kernel_size,
        multires=False,
        last_only=False,
        input_shape=tuple([frames_per_batch] + list(train_dict['X'].shape[2:])),
        norm_method=norm_method)

    print("Training segmentation model. \n")

    conv_gru_model = train_model_conv(
        model=conv_gru_model,
        train_dict=train_dict,
        test_dict=test_dict,
        model_name=conv_gru_model_name,
        log_dir=LOG_DIR,
        optimizer=optimizer,
        transform='watershed', #'deepcell_flat', 
        dilation_radius=1,
        batch_size=batch_size,
        frames_per_batch=frames_per_batch,
        n_epoch=n_epoch,
        model_dir=MODEL_DIR,
        lr_sched=lr_sched,
        rotation_range=180,
        flip=True,
        shear=False,
        zoom_range=(0.8, 1.2))
    
    # Save model
    conv_gru_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(conv_gru_model_name))
    conv_gru_model.save_weights(conv_gru_weights_file)

# Train the models

### Training settings

In [ ]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

fgbg_model_name = 'trackingSEG_cropped_conv_gru_fgbg_model'
conv_model_name = 'trackingSEG_cropped_conv_gru_watershed_model'

n_epoch = 10  # Number of training epochs
test_size = .10  # % of data saved as test
norm_method = 'std'  # data normalization
receptive_field = 61  # should be adjusted for the scale of the data

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# FC training settings
n_skips = 3  # number of skip-connections (only for FC training)
batch_size = 1  # FC training uses 1 image per batch
frames_per_batch = 3

# Transformation settings
transform = 'watershed'
distance_bins = 4
erosion_width = 1  # erode edges

### (1) Train fgbg model

In [ ]:
fgbg_model =  create_and_train_fgbg(lr_sched, train_dict, test_dict, fgbg_model_name, gru_kernel_size=3, gru=True)

In [24]:
watershed_model =  create_and_train_gru(lr_sched, train_dict, test_dict, conv_model_name, gru_kernel_size=3, gru=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 3, 135, 160, 1)    0         
_________________________________________________________________
image_normalization3d_6 (Ima (None, 3, 135, 160, 1)    0         
_________________________________________________________________
reflection_padding3d_4 (Refl (None, 5, 195, 220, 1)    0         
_________________________________________________________________
conv3d_32 (Conv3D)           (None, 5, 192, 217, 32)   544       
_________________________________________________________________
batch_normalization_44 (Batc (None, 5, 192, 217, 32)   128       
_________________________________________________________________
activation_36 (Activation)   (None, 5, 192, 217, 32)   0         
_________________________________________________________________
conv3d_33 (Conv3D)           (None, 5, 190, 215, 32)   9248      
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 3, 135, 160, 5)    0         
_________________________________________________________________
image_normalization3d_8 (Ima (None, 3, 135, 160, 5)    0         
_________________________________________________________________
reflection_padding3d_6 (Refl (None, 5, 195, 220, 5)    0         
_________________________________________________________________
conv3d_48 (Conv3D)           (None, 5, 192, 217, 32)   2592      
_________________________________________________________________
batch_normalization_66 (Batc (None, 5, 192, 217, 32)   128       
_________________________________________________________________
activation_54 (Activation)   (None, 5, 192, 217, 32)   0         
_________________________________________________________________
conv3d_49 (Conv3D)           (None, 5, 190, 215, 32)   9248      
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 3, 135, 160,  0                                            
__________________________________________________________________________________________________
image_normalization3d_5 (ImageN (None, 3, 135, 160,  0           input_6[0][0]                    
__________________________________________________________________________________________________
model_6 (Model)                 (None, 3, 135, 160,  461060      image_normalization3d_5[0][0]    
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 3, 135, 160,  0           image_normalization3d_5[0][0]    
                                                                 model_6[1][0]                    
__________

Epoch 8/10
270/271 [============================>.] - ETA: 1s - loss: 4.7978 - model_6_loss: 1.2002 - model_7_loss: 1.1813 - model_8_loss: 1.1776 - model_9_loss: 1.1732 - model_6_acc: 0.8788 - model_7_acc: 0.8801 - model_8_acc: 0.8806 - model_9_acc: 0.8802
Epoch 00008: saving model to /data/models/trackingSEG_stacked_conv_gru_watershed_model.h5
271/271 [==============================] - 293s 1s/step - loss: 4.7984 - model_6_loss: 1.2004 - model_7_loss: 1.1814 - model_8_loss: 1.1777 - model_9_loss: 1.1735 - model_6_acc: 0.8789 - model_7_acc: 0.8801 - model_8_acc: 0.8807 - model_9_acc: 0.8803 - val_loss: 4.8650 - val_model_6_loss: 1.2107 - val_model_7_loss: 1.1909 - val_model_8_loss: 1.1853 - val_model_9_loss: 1.2127 - val_model_6_acc: 0.8598 - val_model_7_acc: 0.8523 - val_model_8_acc: 0.8495 - val_model_9_acc: 0.8400
Epoch 9/10
270/271 [============================>.] - ETA: 1s - loss: 4.7465 - model_6_loss: 1.1906 - model_7_loss: 1.1706 - model_8_loss: 1.1601 - model_9_loss: 1.1596 - 

### Run the model

#### Make predictions on test data

In [49]:
fgbg_model = feature_net_skip_3D(
        receptive_field=receptive_field,
        n_features=2,
        n_frames=frames_per_batch,
        n_skips=n_skips,
        n_conv_filters=32,
        n_dense_filters=128,
        input_shape=tuple(X_test.shape[1:]),
        gru=True,
        gru_kernel_size=3,
        multires=False,
        last_only=False,
        norm_method=norm_method)

fgbg_model.load_weights('/data/models/trackingSEG_stacked_conv_gru_fgbg_model.h5')


watershed_model = feature_net_skip_3D(
        receptive_field=receptive_field,
        n_skips=n_skips,
        n_features=4,  # (background edge, interior edge, cell interior, background)
        n_frames=frames_per_batch,
        n_conv_filters=32,
        n_dense_filters=128,
        gru=True,
        gru_kernel_size=3,
        multires=False,
        last_only=False,
        input_shape=tuple(X_test.shape[1:]),
        norm_method=norm_method)

watershed_model.load_weights('/data/models/trackingSEG_stacked_conv_gru_watershed_model.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_232 (InputLayer)       (None, 30, 135, 160, 1)   0         
_________________________________________________________________
image_normalization3d_231 (I (None, 30, 135, 160, 1)   0         
_________________________________________________________________
reflection_padding3d_184 (Re (None, 32, 195, 220, 1)   0         
_________________________________________________________________
conv3d_1472 (Conv3D)         (None, 32, 192, 217, 32)  544       
_________________________________________________________________
batch_normalization_2024 (Ba (None, 32, 192, 217, 32)  128       
_________________________________________________________________
activation_1656 (Activation) (None, 32, 192, 217, 32)  0         
_________________________________________________________________
conv3d_1473 (Conv3D)         (None, 32, 190, 215, 32)  9248      
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_234 (InputLayer)       (None, 30, 135, 160, 3)   0         
_________________________________________________________________
image_normalization3d_233 (I (None, 30, 135, 160, 3)   0         
_________________________________________________________________
reflection_padding3d_186 (Re (None, 32, 195, 220, 3)   0         
_________________________________________________________________
conv3d_1488 (Conv3D)         (None, 32, 192, 217, 32)  1568      
_________________________________________________________________
batch_normalization_2046 (Ba (None, 32, 192, 217, 32)  128       
_________________________________________________________________
activation_1674 (Activation) (None, 32, 192, 217, 32)  0         
_________________________________________________________________
conv3d_1489 (Conv3D)         (None, 32, 190, 215, 32)  9248      
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_231 (InputLayer)          (None, 30, 135, 160, 0                                            
__________________________________________________________________________________________________
image_normalization3d_230 (Imag (None, 30, 135, 160, 0           input_231[0][0]                  
__________________________________________________________________________________________________
model_232 (Model)               (None, 30, 135, 160, 460802      image_normalization3d_230[0][0]  
__________________________________________________________________________________________________
concatenate_138 (Concatenate)   (None, 30, 135, 160, 0           image_normalization3d_230[0][0]  
                                                                 model_232[1][0]                  
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_239 (InputLayer)       (None, 30, 135, 160, 5)   0         
_________________________________________________________________
image_normalization3d_238 (I (None, 30, 135, 160, 5)   0         
_________________________________________________________________
reflection_padding3d_190 (Re (None, 32, 195, 220, 5)   0         
_________________________________________________________________
conv3d_1520 (Conv3D)         (None, 32, 192, 217, 32)  2592      
_________________________________________________________________
batch_normalization_2090 (Ba (None, 32, 192, 217, 32)  128       
_________________________________________________________________
activation_1710 (Activation) (None, 32, 192, 217, 32)  0         
_________________________________________________________________
conv3d_1521 (Conv3D)         (None, 32, 190, 215, 32)  9248      
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_236 (InputLayer)          (None, 30, 135, 160, 0                                            
__________________________________________________________________________________________________
image_normalization3d_235 (Imag (None, 30, 135, 160, 0           input_236[0][0]                  
__________________________________________________________________________________________________
model_237 (Model)               (None, 30, 135, 160, 461060      image_normalization3d_235[0][0]  
__________________________________________________________________________________________________
concatenate_141 (Concatenate)   (None, 30, 135, 160, 0           image_normalization3d_235[0][0]  
                                                                 model_237[1][0]                  
__________

In [50]:
# make predictions on testing data
test_images = watershed_model.predict(X_test[:6, :, :, :, :])[-1]
test_images_fgbg = fgbg_model.predict(X_test[:6, :, :, :, :])[-1]

print('watershed transform shape:', test_images.shape)
print('segmentation mask shape:', test_images_fgbg.shape)

watershed transform shape: (6, 30, 135, 160, 4)
segmentation mask shape: (6, 30, 135, 160, 2)


### Watershed post-processing

In [51]:
# Collapse predictions into semantic segmentation mask

argmax_images = []
for i in range(test_images.shape[0]):
    max_image = np.argmax(test_images[i], axis=-1)
    argmax_images.append(max_image)
argmax_images = np.array(argmax_images)
argmax_images = np.expand_dims(argmax_images, axis=-1)

print('watershed argmax shape:', argmax_images.shape)

watershed argmax shape: (6, 30, 135, 160, 1)


In [64]:
# threshold the foreground/background
# and remove background from watershed transform
threshold = 0.95
fg_thresh = test_images_fgbg[..., 1] > threshold

fg_thresh = np.expand_dims(fg_thresh.astype('int16'), axis=-1)
argmax_images_post_fgbg = argmax_images * fg_thresh

In [65]:
# Apply watershed method with the distance transform as seed
from skimage.measure import label
from skimage.morphology import watershed
from skimage.feature import peak_local_max

watershed_images = []
for i in range(argmax_images_post_fgbg.shape[0]):
    image = fg_thresh[i, ..., 0]
    distance = argmax_images_post_fgbg[i, ..., 0]

    local_maxi = peak_local_max(test_images[i, ..., -1], min_distance=15, 
                                exclude_border=False, indices=False, labels=image)

    markers = label(local_maxi)
    segments = watershed(-distance, markers, mask=image)
    watershed_images.append(segments)

watershed_images = np.array(watershed_images)
watershed_images = np.expand_dims(watershed_images, axis=-1)

### Plot the Results

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

index = np.random.randint(low=0, high=6)
frame = np.random.randint(low=0, high=X_test.shape[1])
print('Image number:', index)

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15, 15), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(X_test[index, frame, ..., 0])
ax[0].set_title('Source Image')

ax[1].imshow(test_images_fgbg[index, frame, ..., 1])
ax[1].set_title('Segmentation Prediction')

ax[2].imshow(fg_thresh[index, frame, ..., 0], cmap='jet')
ax[2].set_title('FGBG Threshold {}%'.format(threshold * 100))

ax[3].imshow(argmax_images[index, frame, ..., 0], cmap='jet')
ax[3].set_title('Distance Transform')

ax[4].imshow(argmax_images_post_fgbg[index, frame, ..., 0], cmap='jet')
ax[4].set_title('Distance Transform w/o Background')

ax[5].imshow(watershed_images[index, frame, ..., 0], cmap='jet')
ax[5].set_title('Watershed Segmentation')

fig.tight_layout()
plt.show()
# plt.savefig('test.png')

### Run the Model Iteratively

In [37]:
# Define the Datasets

###### For Deepcell Datasets

# Define data to load (raw images from trk test files)
RAW_BASE_DIR = '/data/npz_data/tracking_benchmark_data/test'

raw_trks_3T3  = os.path.join(RAW_BASE_DIR, '3T3_NIH_test_BData.trks')
raw_trks_HEK  = os.path.join(RAW_BASE_DIR, 'HEK293_generic_test_BData.trks')
raw_trks_HeLa = os.path.join(RAW_BASE_DIR, 'HeLa_S3_test_BData.trks')
raw_trks_RAW  = os.path.join(RAW_BASE_DIR, 'RAW264_generic_test_BData.trks')

raw_trks_files = [raw_trks_3T3, raw_trks_HEK, raw_trks_HeLa]
# raw_trks_files = [raw_trks_3T3, raw_trks_HEK, raw_trks_HeLa, raw_trks_RAW]

# Define a base file name for the output
BASE_NAME = 'batch_'

In [38]:
# Instantiate new models for any image size

from deepcell import model_zoo

def test_gru(X_test):
    fgbg_model = feature_net_skip_3D(
            receptive_field=receptive_field,
            n_features=2,
            n_frames=frames_per_batch,
            n_skips=n_skips,
            n_conv_filters=32,
            n_dense_filters=128,
            input_shape=tuple(X_test.shape[1:]),
            gru=True,
            gru_kernel_size=3,
            multires=False,
            last_only=False,
            norm_method=norm_method)

    fgbg_model.load_weights('/data/models/trackingSEG_cropped_conv_gru_fgbg_model.h5')


    watershed_model = feature_net_skip_3D(
            receptive_field=receptive_field,
            n_skips=n_skips,
            n_features=4,  # (background edge, interior edge, cell interior, background)
            n_frames=frames_per_batch,
            n_conv_filters=32,
            n_dense_filters=128,
            gru=True,
            gru_kernel_size=3,
            multires=False,
            last_only=False,
            input_shape=tuple(X_test.shape[1:]),
            norm_method=norm_method)

    watershed_model.load_weights('/data/models/trackingSEG_cropped_conv_gru_watershed_model.h5')

    return fgbg_model, watershed_model

In [41]:
# Make predictions on test data
from deepcell.utils.tracking_utils import load_trks
from skimage.morphology import remove_small_objects
import pandas as pd
from deepcell import metrics
from skimage.measure import label
from skimage.morphology import watershed
from skimage.feature import peak_local_max

CELL_TYPE_NAME = ['3T3', 'HEK293', 'HeLa']
#CELL_TYPE_NAME = ['3T3', 'HEK293', 'HeLa', 'RAW264']

# threshold the foreground/background
# and remove back ground from edge transform
threshold = 0.5
Lstats_AllDatasets = []
# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for set_num, dataset in enumerate(raw_trks_files):
    # Load the trk file       
    trks = load_trks(dataset)
    lineages, raw, tracked = trks['lineages'], trks['X'], trks['y']
    # Create an array for the new annotations
    y_new = np.zeros(tracked.shape)
    Lstats = []
    # Go through each batch (movie) in each dataset
    # Predict on the raw data
    for j in range(0, len(trks['y']), 3):
        top = min(j+3, len(trks['y']))
        X_test, y_test = trks['X'][j:top, ...], trks['y'][j:top,...]
        print("X_test.shape", X_test.shape)
        fgbg_model, watershed_model = test_gru(X_test)
        test_images = watershed_model.predict(X_test)[-1]
        test_images_fgbg = fgbg_model.predict(X_test)[-1]
        # Postprocessing
        # Collapse predictions into semantic segmentation mask
        argmax_images = []
        for i in range(test_images.shape[0]):
            max_image = np.argmax(test_images[i], axis=-1)
            argmax_images.append(max_image)
        argmax_images = np.array(argmax_images)
        argmax_images = np.expand_dims(argmax_images, axis=-1)
        # threshold the foreground/background
        # and remove background from watershed transform
        fg_thresh = test_images_fgbg[..., 1] > threshold
        fg_thresh = np.expand_dims(fg_thresh.astype('int16'), axis=-1)
        argmax_images_post_fgbg = argmax_images * fg_thresh
        watershed_images = []
        for i in range(argmax_images_post_fgbg.shape[0]):
            image = fg_thresh[i, ..., 0]
            distance = argmax_images_post_fgbg[i, ..., 0]
            local_maxi = peak_local_max(test_images[i, ..., -1], min_distance=15, 
                                        exclude_border=False, indices=False, labels=image)
            markers = label(local_maxi)
            segments = watershed(-distance, markers, mask=image)
            watershed_images.append(segments)
        watershed_images = np.array(watershed_images)
        watershed_images = np.expand_dims(watershed_images, axis=-1)
        print("watershed_images.shape", watershed_images.shape)
        # Accuracy from metrics package
        batch_num = i / 6
        filename = CELL_TYPE_NAME[set_num] + BASE_NAME + '{}'.format(batch_num)
        # Remove small objects from GT for comparison
        small_objects_threshold=100        
        GT_image = remove_small_objects(tracked[j:top, :, :, :, 0].astype('uint16'), min_size=small_objects_threshold)
        print("GT_image.shape", GT_image.shape)
        #GT_image = tracked[batch_num, :, :, :, :].astype('uint16')
        m = metrics.Metrics(model_name = filename)
        m.calc_object_stats(GT_image, watershed_images[:,:,:,:, 0])
        #m.save_to_json(m.output)    
        Lstats.append(m.stats)                
    Lstats_AllDatasets.append(Lstats)
df_3T3    = pd.concat(Lstats_AllDatasets[0]) 
df_HEK293 = pd.concat(Lstats_AllDatasets[1])
df_HeLa   = pd.concat(Lstats_AllDatasets[2])
# df_RAW264 = pd.concat(Lstats_AllDatasets[3])

X_test.shape (3, 30, 154, 182, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_52 (InputLayer)        (None, 30, 154, 182, 1)   0         
_________________________________________________________________
image_normalization3d_51 (Im (None, 30, 154, 182, 1)   0         
_________________________________________________________________
reflection_padding3d_40 (Ref (None, 32, 214, 242, 1)   0         
_________________________________________________________________
conv3d_320 (Conv3D)          (None, 32, 211, 239, 32)  544       
_________________________________________________________________
batch_normalization_440 (Bat (None, 32, 211, 239, 32)  128       
_________________________________________________________________
activation_360 (Activation)  (None, 32, 211, 239, 32)  0         
_________________________________________________________________
conv3d_321 (Conv3D)          (None, 32, 20

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_54 (InputLayer)        (None, 30, 154, 182, 3)   0         
_________________________________________________________________
image_normalization3d_53 (Im (None, 30, 154, 182, 3)   0         
_________________________________________________________________
reflection_padding3d_42 (Ref (None, 32, 214, 242, 3)   0         
_________________________________________________________________
conv3d_336 (Conv3D)          (None, 32, 211, 239, 32)  1568      
_________________________________________________________________
batch_normalization_462 (Bat (None, 32, 211, 239, 32)  128       
_________________________________________________________________
activation_378 (Activation)  (None, 32, 211, 239, 32)  0         
_________________________________________________________________
conv3d_337 (Conv3D)          (None, 32, 209, 237, 32)  9248      
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_51 (InputLayer)           (None, 30, 154, 182, 0                                            
__________________________________________________________________________________________________
image_normalization3d_50 (Image (None, 30, 154, 182, 0           input_51[0][0]                   
__________________________________________________________________________________________________
model_52 (Model)                (None, 30, 154, 182, 460802      image_normalization3d_50[0][0]   
__________________________________________________________________________________________________
concatenate_30 (Concatenate)    (None, 30, 154, 182, 0           image_normalization3d_50[0][0]   
                                                                 model_52[1][0]                   
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_59 (InputLayer)        (None, 30, 154, 182, 5)   0         
_________________________________________________________________
image_normalization3d_58 (Im (None, 30, 154, 182, 5)   0         
_________________________________________________________________
reflection_padding3d_46 (Ref (None, 32, 214, 242, 5)   0         
_________________________________________________________________
conv3d_368 (Conv3D)          (None, 32, 211, 239, 32)  2592      
_________________________________________________________________
batch_normalization_506 (Bat (None, 32, 211, 239, 32)  128       
_________________________________________________________________
activation_414 (Activation)  (None, 32, 211, 239, 32)  0         
_________________________________________________________________
conv3d_369 (Conv3D)          (None, 32, 209, 237, 32)  9248      
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_56 (InputLayer)           (None, 30, 154, 182, 0                                            
__________________________________________________________________________________________________
image_normalization3d_55 (Image (None, 30, 154, 182, 0           input_56[0][0]                   
__________________________________________________________________________________________________
model_57 (Model)                (None, 30, 154, 182, 461060      image_normalization3d_55[0][0]   
__________________________________________________________________________________________________
concatenate_33 (Concatenate)    (None, 30, 154, 182, 0           image_normalization3d_55[0][0]   
                                                                 model_57[1][0]                   
__________

...

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_224 (InputLayer)       (None, 40, 216, 256, 3)   0         
_________________________________________________________________
image_normalization3d_223 (I (None, 40, 216, 256, 3)   0         
_________________________________________________________________
reflection_padding3d_178 (Re (None, 42, 276, 316, 3)   0         
_________________________________________________________________
conv3d_1424 (Conv3D)         (None, 42, 273, 313, 32)  1568      
_________________________________________________________________
batch_normalization_1958 (Ba (None, 42, 273, 313, 32)  128       
_________________________________________________________________
activation_1602 (Activation) (None, 42, 273, 313, 32)  0         
_________________________________________________________________
conv3d_1425 (Conv3D)         (None, 42, 271, 311, 32)  9248      
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_221 (InputLayer)          (None, 40, 216, 256, 0                                            
__________________________________________________________________________________________________
image_normalization3d_220 (Imag (None, 40, 216, 256, 0           input_221[0][0]                  
__________________________________________________________________________________________________
model_222 (Model)               (None, 40, 216, 256, 460802      image_normalization3d_220[0][0]  
__________________________________________________________________________________________________
concatenate_132 (Concatenate)   (None, 40, 216, 256, 0           image_normalization3d_220[0][0]  
                                                                 model_222[1][0]                  
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_229 (InputLayer)       (None, 40, 216, 256, 5)   0         
_________________________________________________________________
image_normalization3d_228 (I (None, 40, 216, 256, 5)   0         
_________________________________________________________________
reflection_padding3d_182 (Re (None, 42, 276, 316, 5)   0         
_________________________________________________________________
conv3d_1456 (Conv3D)         (None, 42, 273, 313, 32)  2592      
_________________________________________________________________
batch_normalization_2002 (Ba (None, 42, 273, 313, 32)  128       
_________________________________________________________________
activation_1638 (Activation) (None, 42, 273, 313, 32)  0         
_________________________________________________________________
conv3d_1457 (Conv3D)         (None, 42, 271, 311, 32)  9248      
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_226 (InputLayer)          (None, 40, 216, 256, 0                                            
__________________________________________________________________________________________________
image_normalization3d_225 (Imag (None, 40, 216, 256, 0           input_226[0][0]                  
__________________________________________________________________________________________________
model_227 (Model)               (None, 40, 216, 256, 461060      image_normalization3d_225[0][0]  
__________________________________________________________________________________________________
concatenate_135 (Concatenate)   (None, 40, 216, 256, 0           image_normalization3d_225[0][0]  
                                                                 model_227[1][0]                  
__________

ResourceExhaustedError: OOM when allocating tensor with shape[192,128,42,27,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node model_227/conv3d_1447/Conv3D}} = Conv3D[T=DT_FLOAT, data_format="NDHWC", dilations=[1, 1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](model_227/conv3d_1447/SpaceToBatchND, model_227/conv3d_1447/Conv3D/ReadVariableOp)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node model_230/tensor_product_366/Tensordot/Shape/_96519}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3716_model_230/tensor_product_366/Tensordot/Shape", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:C

In [42]:
CELL_TYPE_DF = [df_3T3, df_HEK293, df_HeLa]

# CELL_TYPE_DF = [df_3T3, df_HEK293, df_HeLa, df_RAW264]

for cell_type, df in zip(CELL_TYPE_NAME, CELL_TYPE_DF):
    print('\n')
    print(cell_type)
    print('\n')
    # Total number of correct detections and incorrect detections
    correct_det = int(df['correct_detections'].sum())
    incorrect_det = int(df['n_pred'].sum() - df['correct_detections'].sum())

    print('Correct Detections: ', correct_det)
    print('Incorrect Detections: ', incorrect_det)
    
    # Total number of splits, merges, and catastrophes
    splits = df['split'].sum()
    merges = df['merge'].sum()
    catastrophes = df['catastrophe'].sum()
    
    print('Splits: ', splits)
    print('Merges: ', merges)
    print('Catastrophes: ', catastrophes)

    # Average Recall, Precision, and Jaccard Index
    recall = 100 * df['correct_detections'].sum() / df['n_true'].sum()
    precision = 100 * df['correct_detections'].sum() / df['n_pred'].sum()
    jaccard = df['jaccard'].mean()

    print('Recall: ', recall)
    print('Precision: ', precision)
    print('Average Jaccard Index: ', jaccard)

NameError: name 'df_3T3' is not defined

### Make Predictions on ISBI Data for Metrics

In [6]:
###### FOR ISBI Challenge Datasets

# Define data to load (raw images from trk test files)  
RAW_BASE_DIR = '/data/data/ISBI_Tracking_Challenge/tracks'

raw_trks_HeLa = os.path.join(RAW_BASE_DIR, 'ISBI_HeLa_Chal_GT.trks')
raw_trks_MSC  = os.path.join(RAW_BASE_DIR, 'ISBI_MSC_resized_Chal_GT_normalized.trks')
raw_trks_U373 = os.path.join(RAW_BASE_DIR, 'ISBI_U373_Chal_GT.trks')

#raw_trks_files = [raw_trks_HeLa, raw_trks_MSC, raw_trks_U373]
raw_trks_files = [raw_trks_HeLa]

# Define where segmentations will be saved
SEG_BASE_DIR = '/data/track_data/Final_Benchmarks/ISBI_DC_SEG/Watershed/HeLa_segmentations'
                             
DC_seg_HeLa  = os.path.join(SEG_BASE_DIR, 'HeLa/')
DC_seg_MSC   = os.path.join(SEG_BASE_DIR, 'MSC/')
DC_seg_U373  = os.path.join(SEG_BASE_DIR, 'U373/')

#DC_seg_folders = [DC_seg_HeLa, DC_seg_MSC, DC_seg_U373]
DC_seg_folders = [DC_seg_HeLa]

# Define a base file name for the output
BASE_NAME = 'batch_'

In [24]:
# Instantiate new models for any image size

from deepcell import model_zoo

fgbg_model = model_zoo.bn_feature_net_skip_2D(
    n_features=2,  # segmentation mask (is_cell, is_not_cell)
    receptive_field=receptive_field,
    n_skips=n_skips,
    n_conv_filters=32,
    n_dense_filters=128,
    input_shape=(None, None, 1),
    last_only=False)

fgbg_model.load_weights('/data/models/trackingSEG_stacked_conv_gru_fgbg_model.h5')


watershed_model = model_zoo.bn_feature_net_skip_2D(
    fgbg_model=fgbg_model,
    receptive_field=receptive_field,
    n_skips=n_skips,
    n_features=distance_bins,
    n_conv_filters=32,
    n_dense_filters=128,
    last_only=False,
    input_shape=(None, None, 1))

watershed_model.load_weights('/data/models/trackingSEG_stacked_conv_gru_watershed_model.h5')

ValueError: axes don't match array

In [ ]:
# Run Predict for metric information only

CELL_TYPE_NAME = ['HeLa']
threshold = 0.9

from deepcell import metrics
from skimage.measure import label
from skimage import morphology
from skimage.morphology import watershed
from skimage.feature import peak_local_max

import tarfile
import pathlib
import tempfile
import json
import pandas as pd
from io import BytesIO


def load_trks(filename):
    """Load a trk/trks file.

    Args:
        trks_file: full path to the file including .trk/.trks

    Returns:
        A dictionary with raw, tracked, and lineage data
    """
    with tarfile.open(filename, 'r') as trks:

        # numpy can't read these from disk...
        array_file = BytesIO()
        array_file.write(trks.extractfile('raw.npy').read())
        array_file.seek(0)
        raw = np.load(array_file)
        array_file.close()

        array_file = BytesIO()
        array_file.write(trks.extractfile('tracked.npy').read())
        array_file.seek(0)
        tracked = np.load(array_file)
        array_file.close()

    return {'X': raw, 'y': tracked}


Lstats_AllDatasets = []

# Go through each Dataset
for set_num, dataset in enumerate(raw_trks_files):
    # Load the trk file       
    trks = load_trks(dataset)
    raw, tracked = trks['X'], trks['y']
    
    # Create an array for the new annotations
    y_new = np.zeros(tracked.shape)
                
    Lstats_allmovies = []

    # Go through each batch (movie) in each dataset
    for batch_num, movie in enumerate(trks['X']):

        Lstats = []
        # Predict on the raw data
        for frame_num, frame in enumerate(movie):
            
            image = np.expand_dims(frame, axis=0)
             
            test_image = watershed_model.predict(image)[-1]
            test_image_fgbg = fgbg_model.predict(image)[-1]

            # Postprocessing
            # Collapse predictions into semantic segmentation mask        
            max_image = np.argmax(test_image, axis=-1)
            argmax_image = np.expand_dims(max_image, axis=-1)
        
            # threshold the foreground/background
            # and remove background from watershed transform
            fg_thresh = test_image_fgbg[..., 1] > threshold

            fg_thresh = np.expand_dims(fg_thresh.astype('int16'), axis=-1)
            argmax_image_post_fgbg = argmax_image * fg_thresh
        
            # Apply watershed method with the distance transform as seed
            image = fg_thresh[..., 0]
            distance = argmax_image_post_fgbg[..., 0]

            local_maxi = peak_local_max(test_image[..., -1], min_distance=15, 
                                        exclude_border=False, indices=False, labels=image)

            markers = label(local_maxi)
            segments = watershed(-distance, markers, mask=image)
            watershed_image = np.expand_dims(segments, axis=-1)
            
            # Remove small objects from GT for comparison
            small_objects_threshold=50
            GT_image = trks['y'][batch_num, frame_num, :, :, :]
            GT_image = np.expand_dims(GT_image, axis=0)
            GT_image = morphology.remove_small_objects(GT_image.astype('uint16'), min_size=small_objects_threshold)
            
            # Accuracy from metrics package
            filename = CELL_TYPE_NAME[set_num] + BASE_NAME + '{}'.format(batch_num)
            
            m = metrics.Metrics(model_name = filename)
            m.calc_object_stats(GT_image, watershed_image)
            
            # Store results
            y_new[batch_num, frame_num, :, :, :] = watershed_image

            Lstats.append(m.stats)
            
    # Save results of this batch before moving on to the next
    filename = os.path.join(DC_seg_folders[set_num], 'all_batches.trks')
    filename = pathlib.Path(filename)

    with tarfile.open(str(filename), 'w') as trks:
        with tempfile.NamedTemporaryFile() as raw_file:
            np.save(raw_file, raw)
            raw_file.flush()
            trks.add(raw_file.name, 'raw.npy')

        with tempfile.NamedTemporaryFile() as tracked_file:
            np.save(tracked_file, y_new)
            tracked_file.flush()
            trks.add(tracked_file.name, 'tracked.npy')
                
        
        Lstats_allmovies.append(Lstats)
        
    Lstats_AllDatasets.append(Lstats_allmovies)
    
    
#df_HeLa   = pd.concat(Lstats_AllDatasets[0])
df_HeLa = pd.concat([pd.concat(Lstats_AllDatasets[0][0]) , pd.concat(Lstats_AllDatasets[0][1])])


In [10]:
CELL_TYPE_DF = [df_HeLa]

for cell_type, df in zip(CELL_TYPE_NAME, CELL_TYPE_DF):
    print('\n')
    print(cell_type)
    print('\n')
    # Total number of correct detections and incorrect detections
    correct_det = int(df['correct_detections'].sum())
    incorrect_det = int(df['n_pred'].sum() - df['correct_detections'].sum())

    print('Correct Detections: ', correct_det)
    print('Incorrect Detections: ', incorrect_det)
    
    # Total number of splits, merges, and catastrophes
    splits = df['split'].sum()
    merges = df['merge'].sum()
    catastrophes = df['catastrophe'].sum()
    
    print('Splits: ', splits)
    print('Merges: ', merges)
    print('Catastrophes: ', catastrophes)

    # Average Recall, Precision, and Jaccard Index
    recall = 100 * df['correct_detections'].sum() / df['n_true'].sum()
    precision = 100 * df['correct_detections'].sum() / df['n_pred'].sum()
    jaccard = df['jaccard'].mean()

    print('Recall: ', recall)
    print('Precision: ', precision)
    print('Average Jaccard Index: ', jaccard)



HeLa


Correct Detections:  29941
Incorrect Detections:  1134
Splits:  21
Merges:  121
Catastrophes:  3
Recall:  94.24892974061949
Precision:  96.35076427996782
Average Jaccard Index:  0.7919377398470048
